In [1]:
#This performs Hierarchical modeling on a partially-observed birth-death process 0->X->0 with birth parameter A and death parameter B.
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
from scipy.stats import gamma
from scipy.stats import poisson
from scipy.stats import skellam

In [2]:
#import saved data
pd_subsampled_x=pd.read_csv("subsampled.csv")
x= pd_subsampled_x.values
size=np.shape(x)

In [3]:
L=3000      #length of the chain
#I=size[0]    #I-1 is number of subintervals
M=size[1]    #number of individuals
I=35
#initializations
A=np.zeros((L,M))               #birth parameter
B=np.zeros((L,M))               #death parameter
a_A=np.zeros(L+1)               #birth shape hyperparameter
a_B=np.zeros(L+1)               #death shape hyperparameter
b_A=np.zeros(L+1)               #birth scale hyperparameter
b_B=np.zeros(L+1)               #death scale hyperparameter
r=np.zeros((2,I-1,size[1]))     #reaction numbers
sum_x=np.zeros(M)   
var_a_A=2                       #birth shape hyperparameter proposal variance
var_a_B=2                       #death shape hyperparameter proposal variance
tune=300                        #reaction proposal tuning parameter
#initial conditions
a_A[0]=70
b_A[0]=10
a_B[0]=10
b_B[0]=10

In [4]:
def init_reaction(x,s):         #initialize reaction numbers           
    q=np.zeros([2,s-1])
    for i in range(s-1):
        if x[i+1]-x[i]>0:
            q[0,i]=np.floor((4/3)*(x[i+1]-x[i]))
            q[1,i]=np.floor((1/3)*(x[i+1]-x[i]))
        else:
            q[0,i]=-np.floor((1/3)*(x[i+1]-x[i]))
            q[1,i]=-np.floor((4/3)*(x[i+1]-x[i]))
    r=q.astype(int)        
    return r  

def accept_rate_react(x,r,prop,i,A,B,jump,b):        #acceptance rate for reaction numbers
    lambda_prop=1+(prop[0]**2)/b
    lambda_cur=1+(r[0]**2)/b
    prop_like=poisson.pmf(prop[0],A)*poisson.pmf(prop[1],0.5*B*(x[i]+x[i+1]))*skellam.pmf(abs(jump),lambda_prop,lambda_prop)
    current_like=poisson.pmf(r[0],A)*poisson.pmf(r[1],0.5*B*(x[i]+x[i+1]))*skellam.pmf(abs(jump),lambda_cur,lambda_cur)  
    rate=np.minimum(1,prop_like/current_like)
    return rate

def a_lik(A,a,b):           #shape hyperparameter likelihood function 
    sum1=0
    M=len(A)
    for s in range(M):
        sum1+=np.log(A[s])
    sum1*=(a-1)
    q=-M*np.log(math.gamma(a))+M*a*np.log(b)+sum1
    return q

In [5]:
#initialize number of reactions
for s in range(M):
    r[:,:,s]= init_reaction(x[:,s],I)
    sum_x[s]=x[0,s]+x[I-1,s]
    for j in range(I-2):
        sum_x[s]+=2*x[j,s]
    sum_x[s]=0.5*sum_x[s] 

In [ ]:
r=np.load('react.npy')

In [ ]:
#perform sampling       
for i in range(L):
    for s in range(M):
        #sample parameters
        A[i,s]=np.random.gamma(np.sum(r[0,:,s])+a_A[i],(I-1+b_A[i])**(-1))
        B[i,s]=np.random.gamma(np.sum(r[1,:,s])+a_B[i],(sum_x[s]+b_A[i])**(-1))
        #update reaction numbers
        for j in range(I-1):
            r_vprop=np.array([-1,-1])
            lamb=1+(r[0,j,s]**2)/tune                 #tuning parameter suggested in Boy's
            while r_vprop[0]<0 or r_vprop[1]<0:
                jump=np.random.poisson(lamb)-np.random.poisson(lamb)
                r_prop=r[0,j,s]+jump
                r_vprop=[r_prop, r_prop-(x[j+1,s]-x[j,s])]    
            rate=accept_rate_react(x[:,s],r[:,j,s],r_vprop,j,A[i,s],B[i,s],jump,tune) 
            if np.random.uniform(0,1,1)<rate:
                r[:,j,s]=r_vprop      
    #print(r)
    #sample hyperparameters a_A and a_B
    a_Aprop=-1
    while a_Aprop<0:
        a_Aprop=a_A[i]+np.random.normal(0,var_a_A)
    a=a_lik(A[i,:],a_Aprop,b_A[i]) 
    b=a_lik(A[i,:],a_A[i],b_A[i])
    rate=np.minimum(1,np.exp(a-b))
    if np.random.uniform(0,1,1)<rate:    
        a_A[i+1]=a_Aprop
    else:
        a_A[i+1]=a_A[i]
        
    a_Bprop=-1
    while a_Bprop<0:
        a_Bprop=a_B[i]+np.random.normal(0,var_a_B)
    a=a_lik(B[i,:],a_Bprop,b_B[i]) 
    b=a_lik(B[i,:],a_B[i],b_B[i])
    rate=np.minimum(1,np.exp(a-b))
    if np.random.uniform(0,1,1)<rate:    
        a_B[i+1]=a_Bprop
    else:
        a_B[i+1]=a_B[i]    
        
    #sample hyperparameters b_A and b_B
    sum1=np.sum(A[i,:])
    sum2=np.sum(B[i,:])
    b_A[i+1]=np.random.gamma(M*a_A[i+1],sum1**(-1))
    b_B[i+1]=np.random.gamma(M*a_B[i+1],sum2**(-1))
    
    if i%5 == 0:
        print(a_A[i+1],b_A[i+1],"next",b_A[i+1],b_B[i+1])                        

70.0 21.781596012401717 next 21.781596012401717 104.49055214161307
69.74121434957164 24.21243059994483 next 24.21243059994483 77.35098657696292
68.52128334057883 21.14610706828713 next 21.14610706828713 69.81826645298064
70.98945973117264 18.944424065782552 next 18.944424065782552 62.82423476386173
68.97978970922358 17.334474432533902 next 17.334474432533902 63.77589447891084
68.0886544456669 16.65895472346192 next 16.65895472346192 66.12031775730742
72.39999895558465 16.840278560261396 next 16.840278560261396 68.60028531315574
72.94473390643184 15.330977618948877 next 15.330977618948877 61.915185015226825
74.09000960315028 15.28029662005876 next 15.28029662005876 60.5192099995555
79.01483111148744 15.011733529295615 next 15.011733529295615 57.035947141264785
79.59701658567461 15.532522536733437 next 15.532522536733437 54.80375573031134
78.18613307751083 14.445965502769637 next 14.445965502769637 55.226836594397945
77.22577050886476 14.271775000624446 next 14.271775000624446 48.5949603

In [ ]:
#save the simulated values into .csv files
np.save('react.npy',r) 
columns=' '
np.savetxt("A.csv", A, delimiter=",",header=columns)
np.savetxt("B.csv", B, delimiter=",",header=columns)
np.savetxt("a_A.csv", a_A, delimiter=",",header=columns)
np.savetxt("b_A.csv", b_A, delimiter=",",header=columns)
np.savetxt("a_B.csv", a_B, delimiter=",",header=columns)
np.savetxt("b_B.csv", b_B, delimiter=",",header=columns)

In [ ]:
1+r[0]**2/200

In [ ]:
np.random.poisson(1+400)-np.random.poisson(1+400)

In [ ]:
r

In [ ]:
abs(4-5)

In [ ]:
jump

In [ ]:
plt.hist(b_A[0:i])